In [8]:
import pandas as pd
import requests
import io


In [9]:
# URLs for EMass 2017-11-11
# http://www.coolrunning.com/results/17/ma.shtml#11

# Format is: URL, results format, division

# 0 = Default format: Place, number, name, ...
# 1 = Place, name, ...

urls_boys = [('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set4.shtml', 0, 1),  ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set5.shtml', 0, 2),  ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set6.shtml', 0, 3),  ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set4.shtml', 0, 4),  ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set5.shtml', 0, 5),  ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set6.shtml', 1, 6)]

urls_girls = [('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set1.shtml', 1, 1), ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set2.shtml', 1, 2), ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set3.shtml', 0, 3), ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set1.shtml', 0, 4), ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set2.shtml', 0, 5), ('http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set3.shtml', 1, 6)]

In [10]:
for url, f, div in urls_boys:
    print(url, f, div)

http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set4.shtml 0 1
http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set5.shtml 0 2
http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_set6.shtml 0 3
http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set4.shtml 0 4
http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set5.shtml 0 5
http://www.coolrunning.com/results/17/ma/Nov11_MIAAEa_1_set6.shtml 1 6


In [11]:
def get_fixed_width_table(webpage, start_string, end_string, first_col_width=3):
    '''
    webpage is a multi-line text object, typically a web page URL
    
    This function ignores any text before it sees the first occurrence of start_string, 
    and any text after end_string
    '''
    
    #print(webpage)
    
    str_tbl = ''
    
    start_flag = False
    end_flag = False
    
    for l in webpage.splitlines():
        
        #print('Line: ' + l)
        
        if end_flag:
            break
        
        if not (start_flag):
            
            # Look for the start string
            if l.find(start_string) >= 0:
                start_flag = True
    
        else:
            
            if l.find(end_string) >= 0:
                end_flag = True
            else:
                # Grab data, if clean:
                if l[0:first_col_width].strip().isdigit():
                    str_tbl = str_tbl + l + '\n'
    
    return str_tbl

In [12]:
# Build the master dataFrame for boys

df_list = []

for u, layout, div in urls_boys:
    results_page = requests.get(u)
    results_table = get_fixed_width_table(results_page.text, '====', 'Team Scores')

    # Convert a string to a string we can send to read_fwf
    io_results_table = io.StringIO(results_table)

    if layout == 1:
        fields=[(0, 3), (0, 0), (4, 30), (30, 32), (33, 54), (0, 0), (55,65), (66, 69)]
    else:
        # Default
        fields=[(0, 3), (4, 9), (10, 30), (30, 32), (33, 56), (57, 65), (66,75), (76, 80)]
  
    df_results = pd.read_fwf(io_results_table, colspecs=fields, names = ['Place', 'Number', 'Name', 'Grade', 'School', 'Pace', 'TimeString', 'Score'] )
    df_results['Division'] = div
    df_list.append(df_results)
    
#    print(df_results.head(10))
    
df_results_boys = pd.concat(df_list)


In [13]:
df_list = []

for u, layout, div in urls_girls:
    results_page = requests.get(u)
    results_table = get_fixed_width_table(results_page.text, '====', 'Team Scores')

    # Convert a string to a string we can send to read_fwf
    io_results_table = io.StringIO(results_table)

    if layout == 1:
        fields=[(0, 3), (0, 0), (4, 30), (30, 32), (33, 54), (0, 0), (55,65), (66, 69)]
    else:
        # Default
        fields=[(0, 3), (4, 9), (10, 30), (30, 32), (33, 56), (57, 65), (66,75), (76, 80)]
  
    df_results = pd.read_fwf(io_results_table, colspecs=fields, names = ['Place', 'Number', 'Name', 'Grade', 'School', 'Pace', 'TimeString', 'Score'] )
    df_results['Division'] = div
    df_list.append(df_results)
    
    print(df_results.head(10))
    
df_results_girls = pd.concat(df_list)

   Place  Number                   Name  Grade        School  Pace TimeString  \
0      1     NaN        Barry, Caroline     12  Newton South   NaN   18:26.95   
1      2     NaN      Sheffield, Olivia     12        Lowell   NaN   18:36.07   
2      3     NaN          Gavish, Einat     12  Newton North   NaN   18:49.68   
3      4     NaN        MvMahon, Vivian      9     Brookline   NaN   19:06.91   
4      5     NaN        Dunbury, Nicole      9      Brockton   NaN   19:16.84   
5      6     NaN         Fragione, Sara     11       Methuen   NaN   19:19.38   
6      7     NaN            Haney, Nell     12       Needham   NaN   19:20.97   
7      8     NaN  Barrett-Tzannes, Maya     12     Lexington   NaN   19:22.92   
8      9     NaN         Ahern, Lindsey     11        Lowell   NaN   19:30.96   
9     10     NaN       Clermont, Nicole     12      Franklin   NaN   19:34.12   

   Score  Division  
0    1.0         1  
1    2.0         1  
2    3.0         1  
3    4.0         1  
4  

In [14]:
print(df_results_boys.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035 entries, 0 to 161
Data columns (total 9 columns):
Place         1035 non-null int64
Number        873 non-null object
Name          1035 non-null object
Grade         1028 non-null float64
School        1035 non-null object
Pace          873 non-null object
TimeString    1035 non-null object
Score         1017 non-null float64
Division      1035 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 80.9+ KB
None


In [15]:
#df_results_boys = df_results_boys.sort_values('Time')

df_results_boys['Time'] = pd.to_datetime(df_results_boys['TimeString'], format='%M:%S.%f').dt.time
df_results_boys = df_results_boys.sort_values('Time')

df_results_girls['Time'] = pd.to_datetime(df_results_girls['TimeString'], format='%M:%S.%f').dt.time
df_results_girls = df_results_girls.sort_values('Time')


In [16]:
df_results_boys.head(20)

,Place,Number,Name,Grade,School,Pace,TimeString,Score,Division,Time
0,1,#2337,"Shelgren, Tristan",12.0,Saint John's Prep,4:54.5,15:14.75,1.0,1,00:15:14.750000
1,2,#2156,"Mah, Andrew",12.0,Newton North,4:56.7,15:21.76,2.0,1,00:15:21.760000
2,3,#1496,"Aramburu, Lucas",11.0,Brookline,5:05.0,15:47.54,3.0,1,00:15:47.540000
3,4,#2154,"Horsbugh, Ian",10.0,Newton North,5:05.7,15:49.52,4.0,1,00:15:49.520000
4,5,#2137,"McLeod, Cameron",11.0,Needham,5:06.1,15:50.98,5.0,1,00:15:50.980000
0,1,#1297,"Oosting, Ryan",11.0,Arlington,5:06.3,15:51.52,1.0,2,00:15:51.520000
1,2,#2537,"Weiss, Thomas",12.0,Wellesley,5:07.8,15:56.18,2.0,2,00:15:56.180000
5,6,#2561,"Dolan, Jason",12.0,Westford Academy,5:07.8,15:56.29,6.0,1,00:15:56.290000
2,3,#1596,"McClure, Timothy",12.0,Concord-Carlisle,5:08.3,15:57.62,3.0,2,00:15:57.620000
0,1,#2140,"Acquaviva, Sam",11.0,Newburyport,5:09.0,15:59.78,1.0,4,00:15:59.780000


In [17]:
# This is for convenience when summing runners' relative and overall places
df_results_boys["tmpCnt"] = 1
df_results_boys["RunnerNumber"] = df_results_boys["tmpCnt"].groupby(df_results_boys['School']).cumsum()
df_results_boys["OverallPlace"] = df_results_boys["tmpCnt"].cumsum()

df_results_girls["tmpCnt"] = 1
df_results_girls["RunnerNumber"] = df_results_girls["tmpCnt"].groupby(df_results_girls['School']).cumsum()
df_results_girls["OverallPlace"] = df_results_girls["tmpCnt"].cumsum()


In [23]:
df_results_boys.head(25)

,Place,Number,Name,Grade,School,Pace,TimeString,Score,Division,Time,tmpCnt,RunnerNumber,OverallPlace
0,1,#2337,"Shelgren, Tristan",12.0,Saint John's Prep,4:54.5,15:14.75,1.0,1,00:15:14.750000,1,1,1
1,2,#2156,"Mah, Andrew",12.0,Newton North,4:56.7,15:21.76,2.0,1,00:15:21.760000,1,1,2
2,3,#1496,"Aramburu, Lucas",11.0,Brookline,5:05.0,15:47.54,3.0,1,00:15:47.540000,1,1,3
3,4,#2154,"Horsbugh, Ian",10.0,Newton North,5:05.7,15:49.52,4.0,1,00:15:49.520000,1,2,4
4,5,#2137,"McLeod, Cameron",11.0,Needham,5:06.1,15:50.98,5.0,1,00:15:50.980000,1,1,5
0,1,#1297,"Oosting, Ryan",11.0,Arlington,5:06.3,15:51.52,1.0,2,00:15:51.520000,1,1,6
1,2,#2537,"Weiss, Thomas",12.0,Wellesley,5:07.8,15:56.18,2.0,2,00:15:56.180000,1,1,7
5,6,#2561,"Dolan, Jason",12.0,Westford Academy,5:07.8,15:56.29,6.0,1,00:15:56.290000,1,1,8
2,3,#1596,"McClure, Timothy",12.0,Concord-Carlisle,5:08.3,15:57.62,3.0,2,00:15:57.620000,1,1,9
0,1,#2140,"Acquaviva, Sam",11.0,Newburyport,5:09.0,15:59.78,1.0,4,00:15:59.780000,1,1,10


In [22]:
# Figure out scores

df_results_boys[df_results_boys["RunnerNumber"] < 6].groupby('School').agg({'OverallPlace':'sum'}).sort_values('OverallPlace').head(25)


,OverallPlace
School,
Lowell,150
Wellesley,177
Concord-Carlisle,236
Lexington,258
Newton North,262
Needham,306
Durfee,354
Wakefield,382
Pembroke,408


In [24]:
df_results_boys[(df_results_boys['School'] == 'Lowell') | (df_results_boys['School'] == 'Wellesley') | (df_results_boys['School'] == 'Concord-Carlisle') ]

,Place,Number,Name,Grade,School,Pace,TimeString,Score,Division,Time,tmpCnt,RunnerNumber,OverallPlace
1,2,#2537,"Weiss, Thomas",12.0,Wellesley,5:07.8,15:56.18,2.0,2,00:15:56.180000,1,1,7
2,3,#1596,"McClure, Timothy",12.0,Concord-Carlisle,5:08.3,15:57.62,3.0,2,00:15:57.620000,1,1,9
3,4,#2531,"Cervon, Matthew",12.0,Wellesley,5:09.1,16:00.32,4.0,2,00:16:00.320000,1,2,11
5,6,#1594,"Kessler, Zander",12.0,Concord-Carlisle,5:09.3,16:00.97,6.0,2,00:16:00.970000,1,2,14
7,8,#1904,"Fitzpatrick, Colin",12.0,Lowell,5:09.8,16:02.34,8.0,1,00:16:02.340000,1,1,16
9,10,#1910,"Pinales, Jadiel",11.0,Lowell,5:11.6,16:07.88,10.0,1,00:16:07.880000,1,2,19
14,15,#1906,"Gibson, Nasir",12.0,Lowell,5:14.4,16:16.72,15.0,1,00:16:16.720000,1,3,34
14,15,#2538,"Zarle, Colby",11.0,Wellesley,5:14.5,16:17.11,15.0,2,00:16:17.110000,1,3,37
15,16,#1905,"Galarza, Chris",12.0,Lowell,5:14.8,16:17.84,16.0,1,00:16:17.840000,1,4,40
16,17,#1911,"Rivera, Carlos",12.0,Lowell,5:14.9,16:18.18,17.0,1,00:16:18.180000,1,5,41


In [25]:
# Figure out scores

df_results_girls[df_results_girls["RunnerNumber"] < 6].groupby('School').agg({'OverallPlace':'sum'}).sort_values('OverallPlace').head(25)

,OverallPlace
School,
Concord-Carlisle,219
Tewksbury,236
Wellesley,239
Tri-County Rvt,267
Milton,276
Marthas Vineyard,283
Billerica,296
Lowell,328
Natick,355


In [26]:
df_results_girls[(df_results_girls['School'] == 'Tewksbury') | (df_results_girls['School'] == 'Wellesley') | (df_results_girls['School'] == 'Concord-Carlisle') ]

,Place,Number,Name,Grade,School,Pace,TimeString,Score,Division,Time,tmpCnt,RunnerNumber,OverallPlace
1,2,NaN,"Donahue, Margaret",11.0,Wellesley,NaN,18:30.98,2.0,2,00:18:30.980000,1,1,5
1,2,#1060,"Sessa, Rachel",12.0,Tewksbury,6:01.9,18:44.28,2.0,3,00:18:44.280000,1,1,9
3,4,NaN,"Reichheld, Sarah",11.0,Concord-Carlisle,NaN,18:53.80,4.0,2,00:18:53.800000,1,1,14
4,5,#1056,"Paige, Makayla",9.0,Tewksbury,6:11.7,19:14.75,5.0,3,00:19:14.750000,1,2,29
5,6,#1059,"Robinson, Lily",11.0,Tewksbury,6:11.8,19:15.00,6.0,3,00:19:15,1,3,30
6,7,NaN,"Andrews, Loren",12.0,Wellesley,NaN,19:18.41,7.0,2,00:19:18.410000,1,2,32
8,9,NaN,"Laurent, Eline",12.0,Concord-Carlisle,NaN,19:25.01,9.0,2,00:19:25.010000,1,2,43
10,11,#1058,"O'Leary, Holly",12.0,Tewksbury,6:16.7,19:30.15,11.0,3,00:19:30.150000,1,4,51
14,15,NaN,"Bergman, Iris",9.0,Concord-Carlisle,NaN,19:31.17,15.0,2,00:19:31.170000,1,3,53
15,16,NaN,"Su, Ellen",11.0,Concord-Carlisle,NaN,19:31.48,16.0,2,00:19:31.480000,1,4,54


In [ ]:
df_results_boys[df_results_boys["RunnerNumber"] == 6]